In [1]:
import pandas as pd
from tqdm import tqdm
import requests
import urllib
import random
from multiprocessing import Pool
import matplotlib.pyplot as plt
from collections import Counter
import asyncio
import aiohttp
import urllib.parse

In [2]:
async def get_media(usage_key,session):
    url_media = f'https://api.gbif.org/v1/species/{usage_key}/media'
    async with session.get(url_media) as response:
        response_json = await response.json()
        return response_json
async def get_name(usage_key,session):
    url_media = f'https://api.gbif.org/v1/species/{usage_key}/name'
    async with session.get(url_media) as response:
        response_json = await response.json()
        return response_json

In [3]:
async def print_media(usage_key):
    async with aiohttp.ClientSession() as session:
        media_responses = await get_media(usage_key,session)
        name_response = await get_name(usage_key,session)
    if not media_responses['results']:
        print('No media for',name_response['canonicalName'])
    for result in media_responses['results']:
        print(name_response['canonicalName'], '->' ,result['identifier'])

In [4]:
def sync_print(usage_key):
    asyncio.gather(print_media(usage_key))

In [5]:
sync_print(2930137)

Solanum lycopersicum -> https://images.naturalis.nl/original/125876_tomaat_dsc_8045-1.jpg
Solanum lycopersicum -> https://images.naturalis.nl/original/125880_tomaat_dsc_8253-1.jpg
Solanum lycopersicum -> https://images.naturalis.nl/original/125879_tomaat_dsc_8051-1.jpg
Solanum lycopersicum -> https://images.naturalis.nl/original/126328_tomaat_4.jpg
Solanum lycopersicum -> https://images.naturalis.nl/original/126326_tomaat_5.jpg
Solanum lycopersicum -> https://images.naturalis.nl/original/125877_tomaat_dsc_8046-1.jpg
Solanum lycopersicum -> https://images.naturalis.nl/original/125878_tomaat_dsc_8048-1.jpg
Solanum lycopersicum -> https://zenodo.org/record/8293824/files/figure.png


# Templates

In [6]:
def get_question_1(image_path, correct_answer, options):
    question =  f'What is the scientific name of this plant?'
    answer = f'{correct_answer}'
    difficulty = 2
    return {'image_path': image_path, 'question': question, 'options': options, 'correct_answer': answer, 'difficulty': difficulty}

def get_question_2(image_path, correct_answer, language, options):
    question =  f'What is the common name in {language} of this plant?'
    answer = f'{correct_answer}'
    difficulty = 2
    return {'image_path': image_path, 'question': question, 'options': options, 'correct_answer': answer, 'difficulty': difficulty}

def get_question(image_path, correct_answer, options, template_dict):
    template = template_dict['template']
    question = template.format(**template_dict.get('kwargs'))
    difficulty = template_dict.get('difficulty', 0)
    answer = f'{correct_answer}'
    return {'image_path': image_path, 'question': question, 'options': options, 'correct_answer': answer, 'difficulty': difficulty}

template_dict_1 = {
    'template': 'What is the scientific name of this plant?',
    'difficulty': 2
}

template_dict_2 = {
    'template': 'What is the common name of this plant in {language}?',
    'difficulty': 3,
    'kwargs': {'language': 'English'}
}

template_dict_3 = {
    'template': 'Which genus does this plant belong to?',
    'difficulty': 1,
}

template_dict_4 = {
    'template': 'Which of the following is a major pest of this plant?',
    'difficulty': 1,
}



template_2 = 'What is the common name in {language} of this plant?'
template_3 = 'What genus does this plant '
    